In [130]:
from sympy import sympify

def calculator(expression):
  try:
    result = sympify(expression)
    return str(result)
  except Exception as e:
    return str(e)

In [131]:
calculator(1*2*3*4*5)

'120'

In [132]:
import re

def extract_action_and_input(text):
  action = re.search(r"Action: (.*)", text)
  action_input = re.search(r"Action Input: (.*)", text)
  return action.group(1).strip() if action else None, action_input.group(1).strip() if action else None

extract_action_and_input("""
Thought: To calculate the square root of 144, I can use the math library in Python or a calculator.

Action: Calculator
Action Input: sqrt(144)
""")

('Calculator', 'sqrt(144)')

In [133]:
from openai import OpenAI
from google.colab import userdata


client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get('OPENROUTER_API_KEY'),
)

user_prompt = "Calculate the square root of 144"

chat_history = [
          {
      "role": "system",
      "content": """
      You have access to the following tools:
      Calculator: Use this when you want to do math. Use SymPy expressions, not math function, e.g 2 + 2

      Use the following format:

      Question: the input question you must answer
      Thought: you should always think what to do
      Action: the action to take, should be one of [Calculator]
      Action Input: the input to the action
      Observation: result of the action
      ...(the Thought/Action/Observation can repeat any number of times)
      Thought: I now know the final answer!
      Final Answer: the final answer to the original input question
      """
    },
    {
      "role": "user",
      "content":f"Question: {user_prompt}"
    }
  ]

while True:
  completion = client.chat.completions.create(
    model="meta-llama/llama-3.2-90b-vision-instruct:free",
    messages=chat_history,
    stop = ["Observation:"]
  )

  response_text = completion.choices[0].message.content

  print(response_text)

  action, action_input = extract_action_and_input(response_text)


  # We want to see if the LLM took an action
  if action == 'Calculator':
    action_result= calculator(action_input)
    print(f"Observation: {action_result}")
    chat_history.extend([
        {"role":"assistant", "content" : response_text},
        {"role":"user", "content": f"Observation: {action_result}" }
    ])
  else:
    break

Thought: To calculate the square root of 144, I need to use a mathematical operation that will give me the number that, when multiplied by itself, equals 144.

Action: Calculator
Action Input: sympy.sqrt(144)

Observation: 'Symbol' object has no attribute 'sqrt'
Thought: It seems like the sqrt function is not available in the SymPy calculator. I need to find another way to calculate the square root of 144. I know that in SymPy, the square root of a number can be calculated by raising the number to the power of 1/2.

Action: Calculator
Action Input: 144 ** (1/2)
Observation: 12
Thought: I now know the final answer!

Final Answer: The final answer is 12.


In [134]:
 print(completion.choices[0].message)

ChatCompletionMessage(content='Thought: I now know the final answer!\n\nFinal Answer: The final answer is 12.', refusal='', role='assistant', audio=None, function_call=None, tool_calls=None)


In [135]:
print(response_text)

Thought: I now know the final answer!

Final Answer: The final answer is 12.


In [136]:
chat_history.extend([
    {"role":"assistant", "content" : response_text},
    {"role":"user", "content": "Observation: The knife did not work! A tree has fallen down between you and presents" }
    ])

In [137]:
chat_history

[{'role': 'system',
  'content': '\n      You have access to the following tools:\n      Calculator: Use this when you want to do math. Use SymPy expressions, not math function, e.g 2 + 2  \n\n      Use the following format:\n\n      Question: the input question you must answer\n      Thought: you should always think what to do\n      Action: the action to take, should be one of [Calculator]\n      Action Input: the input to the action\n      Observation: result of the action\n      ...(the Thought/Action/Observation can repeat any number of times)\n      Thought: I now know the final answer! \n      Final Answer: the final answer to the original input question\n      '},
 {'role': 'user', 'content': 'Question: Calculate the square root of 144'},
 {'role': 'assistant',
  'content': 'Thought: To calculate the square root of 144, I need to use a mathematical operation that will give me the number that, when multiplied by itself, equals 144.\n\nAction: Calculator\nAction Input: sympy.sqrt

In [138]:
completion = client.chat.completions.create(
  model="meta-llama/llama-3.2-90b-vision-instruct:free",
  messages=chat_history
)
response_text = completion.choices[0].message.content

In [139]:
response_text

'Thought: This is not relevant to the math problem we were solving. The square root of 144 is still the same.\n\nFinal Answer remains the same: The final answer is 12.'